### Import and definitions

In [ ]:
import random
import pyttsx3
import itertools

In [ ]:
MAX_HAND_CARDS = 3

In [ ]:
class Card:
    def __init__(self, value, seed):
        self.__s = seed
        self.__v = value
    
    def __repr__(self):
        return str(self.__v) + str(self.__s)
    
    @property
    def s(self):
        return self.__s
    
    @property
    def v(self):
        return self.__v

In [ ]:
class Deck:
    seeds = ['d', 's', 'b', 'c']
    cards_per_seed = 10
    
    def __init__(self):
        self.__cards = []
        for s in self.seeds:
            for i in range(1,self.cards_per_seed+1):
                self.__cards.append(Card(i, s))
            
    def __len__(self):
        return len(self.__cards)
    
    def shuffle(self):
        return random.shuffle(self.__cards)
        
    def pop(self):
        return self.__cards.pop()

### Game functions

In [ ]:
def check_pick_rules(table, tbl_indexes, picking_card):
    
    # check if picking card sums up equal to the desired cards
    cards = [c for ind, c in enumerate(table) if ind in tbl_indexes]
    if picking_card.v != sum(c.v for c in cards):
        return False
    
    # check if there is an equal value card instead of summing
    if len(cards) > 1 and len([c for c in table if c.v == picking_card.v]):
        return False
    
    return True


def check_scopa(table, deck, pl1_cards, pl2_cards):
    # scopa if table is empty but there still some card either in the deck or in players' hands
    return len(table) == 0 and (len(deck) != 0 or (len(pl1_cards)+len(pl2_cards) != 0))


def count_points(deck):
    points = 0
    # carte
    tmpDeck = Deck()
    points += 1 if (len(deck) > len(tmpDeck)/2) else 0
    # denari
    points += 1 if (len([c for c in deck if c.s == 'd']) > 5) else 0 
    # settebello
    points += 1 if len([c for c in deck if c.v == 7 and c.s == 'd']) else 0
    # primiera
    primiera_order = [7, 6, 1, 5, 4, 3, 2, 10, 9, 8]
    primiera_val = [21, 18, 16, 15, 14, 13, 12, 10, 10, 10]
    primiera_pts_mine = 0
    primiera_pts_other = 0
    for s in Deck.seeds:
        cards_per_seed = [c.v for c in deck if c.s == s]
        primiera_pts_mine += primiera_val[min([i for i, v in enumerate(primiera_order) if v in cards_per_seed])]
        primiera_pts_other += primiera_val[min([i for i, v in enumerate(primiera_order) if v not in cards_per_seed])]
    print("Primieria: mine=" + str(primiera_pts_mine) + " - other=" + str(primiera_pts_other))
    points += 1 if primiera_pts_mine > primiera_pts_other else 0
    return points

In [ ]:
def give_cards(deck, player_cards, num):
    if len(deck):
        for i in range(0, num):
            player_cards.append(deck.pop())
        return True
    return False
    
    
def pick_cards(table, tbl_indexes, player_cards, pl_index):
    # check indexes
    if pl_index >= len(player_cards) or len([i for i in tbl_indexes if i >= len(table)]):
        return []
    # check rules
    cards = []
    picking_card = player_cards[pl_index]
    if check_pick_rules(table, tbl_indexes, picking_card) == True:
        # pick cards
        cards = [c for i,c in enumerate(table) if i in tbl_indexes]
        for i in sorted(tbl_indexes, reverse=True):
            del table[i]
        cards.append(player_cards.pop(pl_index))
    return cards
    
    
def draw_card(table, player_cards, index):
    if index < len(player_cards):
        table.append(player_cards.pop(index))

### HM Interface

In [ ]:
def parse_play(play_str):
    play = {}
    cmd = play_str.split(':')[0]
    if cmd == "PRENDO":
        play['type'] = cmd
        play['pick_options'] = [list(map(int,play_str.split(':')[1].split(','))), list(map(int,play_str.split(':')[2].split(',')))]
    elif cmd == "CALO":
        play['type'] = cmd
        play['pick_options'] = [[], list(map(int,play_str.split(':')[1].split(',')))]
    else:
        print("Comando invalido. Possibili comandi sono: PRENDO:t1,t2,...:c oppure CALO:c")
    print("PARSE PLAY: " + str(play))
    return play


def print_status(table, my_cards, opp_cards):
    print("Table: " + str(table) + " - My Cards: " + str(my_cards) + " - Your Cards: " + str(opp_cards))
    
def init_voice():
    engine = pyttsx3.init()
    voices = engine.getProperty('voices')
    engine.setProperty('voice', voices[0].id)
    return engine
    
def say(engine, str):
    print(str)
    engine.say(str)
    engine.runAndWait()

### IntelliPlay

In [ ]:
# TODO: check_pick_option has an error ==> valid result like PRENDO [1c] con 5d
def check_pick_option(table, my_cards):
    # check if I have a card equal to one on the table
    for my_i, my_c in enumerate(my_cards):
        for t_i, t_c in enumerate(table):
            if my_c.v == t_c.v:
                return [[t_i],[my_i]]
    
    # check if I can make a sum
    if len(table) > 1:
        for my_i, my_c in enumerate(my_cards):
            # check for all combinations of cards with more than 1 card
            for l in range(1,len(table)+1):
                for comb in itertools.combinations(table, l):
                    if my_c.v == sum(c.v for c in comb):
                        return [[i for i in range(len(table)) if table[i] in comb], [my_i]]

    # cannot pick anything, return an empty set
    return []

### Main

In [ ]:
# init text-to-speech
voice = init_voice()

In [ ]:
# create and the deck
deck = Deck()
deck.shuffle()

my_cards = []
my_deck = []
my_points = 0
opp_cards = []
opp_deck = []
opp_points = 0
table = []
last_pick_deck = my_deck

loop_cnt = 0
while(give_cards(deck, opp_cards, 3)):
    say(voice, "Do le carte")
    give_cards(deck, my_cards, 3)
    if len(deck) == 0:
        say(voice, "Ultima mano")
    
    if loop_cnt == 0:
        give_cards(deck, table, 4)
    
    play_cnt = 0
    while play_cnt < 3:
        # make opponent play (TODO...)
        # how to make this interactive ???
        play = {}
        while len(play) == 0:
            print_status(table, my_cards, opp_cards)
            play_str = input("cosa vuoi fare?")
            play = parse_play(play_str)
        if play['type'] == "PRENDO":
            opp_deck += pick_cards(table, play['pick_options'][0], opp_cards, play['pick_options'][1][0])
            last_pick_deck = opp_deck
        else:
            draw_card(table, opp_cards, play['pick_options'][1][0])

        # check for "scopa"
        opp_points += check_scopa(table, deck, my_cards, opp_cards)
        print_status(table, my_cards, opp_cards)
        
        # make my play
        pick_options = check_pick_option(table, my_cards)
        if len(pick_options):
            say(voice, "Tocca a me: PRENDO " + str([c for i,c in enumerate(table) if i in pick_options[0]]) + " con " + str(my_cards[pick_options[1][0]]))
            my_deck += pick_cards(table, pick_options[0], my_cards, pick_options[1][0])
            last_pick_deck = my_deck
        else:
            random_draw = random.randrange(0,len(my_cards))
            say(voice, "Tocca a me: CALO " + str(my_cards[random_draw]))
            draw_card(table, my_cards, random_draw)
        
        # check for "scopa"
        my_points += check_scopa(table, deck, my_cards, opp_cards)
        
        # increment play_cnt
        play_cnt += 1
        
    # increment loop count
    loop_cnt += 1
    
# collect cards on the table at the last hand
last_pick_deck += table
table.clear()
        
my_points += count_points(my_deck)
opp_points += count_points(opp_deck)

if my_points > opp_points:
    say(voice, "Ho vinto io " + str(my_points) + " punti a " + str(opp_points) + ". Bella partita, grazie!")
elif my_points < opp_points:
    say(voice, "Grande, hai vinto! " + str(opp_points) + " punti a " + str(my_points) + ". Bella partita, grazie!")
else:
    say(voice, "Abbiamo pareggiato: " + str(opp_points) + " a " + str(my_points) + ". Bella partita, grazie!")

In [ ]:
[table, my_deck, opp_deck]